In [5]:
import numpy as np
import matplotlib.pyplot as plt

# Pre-Lab

In [8]:
def lgwt(N, a, b):
    x, w = np.polynomial.legendre.leggauss(N)
    x = 0.5*(x + 1)*(b - a) + a
    w = w * 0.5*(b - a)
    return x, w

In [9]:
def eval_composite_trap(M, a, b, f):
    x = np.linspace(a, b, M)
    h = (b - a) / (M - 1)
    I_hat = h * (0.5 * f(x[0]) + np.sum(f(x[1:-1])) + 0.5 * f(x[-1]))
    return I_hat, x, None

In [10]:
def eval_composite_simpsons(M, a, b, f):
    if M % 2 == 0:
        raise ValueError("M must be odd for Simpson's rule.")
    x = np.linspace(a, b, M)
    h = (b - a) / (M - 1)
    # Simpson's rule approximation:
    I_hat = h/3 * (f(x[0]) + f(x[-1]) +
                   4 * np.sum(f(x[1:-1:2])) +
                   2 * np.sum(f(x[2:-1:2])))
    return I_hat, x, None

In [11]:
def eval_gauss_quad(M, a, b, f):
    x, w = lgwt(M, a, b)
    I_hat = np.sum(f(x) * w)
    return I_hat, x, w

In [12]:
def adaptive_quad(a, b, f, tol, M, method):

    maxit = 50
    left_p = np.zeros(maxit)
    right_p = np.zeros(maxit)
    s = np.zeros(maxit)
    left_p[0] = a
    right_p[0] = b
    s[0], x, _ = method(M, a, b, f)
    X = [x]
    j = 1
    I = 0
    nsplit = 1
    while j < maxit:
        c = 0.5 * (left_p[j-1] + right_p[j-1])
        s1, x_left, _ = method(M, left_p[j-1], c, f)
        s2, x_right, _ = method(M, c, right_p[j-1], f)
        X.append(x_left)
        X.append(x_right)
        if np.abs(s1 + s2 - s[j-1]) > tol:
            left_p[j] = left_p[j-1]
            right_p[j] = c
            s[j] = s1
            left_p[j-1] = c
            s[j-1] = s2
            j += 1
            nsplit += 1
        else:
            I += s1 + s2
            j -= 1
            if j == 0:
                j = maxit
    all_nodes = np.concatenate(X)
    return I, np.unique(all_nodes), nsplit

In [15]:
def integrand(x):
    return np.sin(1/x)

a = 0.1
b = 2.0
tol = 1e-3

M = 5

methods = {
    "Composite Trapezoidal": eval_composite_trap,
    "Composite Simpson's": eval_composite_simpsons,
    "Gaussian Quadrature": eval_gauss_quad
}

results = {}

print("Adaptive Quadrature Results for sin(1/x) on [0.1, 2] with tol=1e-3 and M=5 nodes:")
for method_name, method_func in methods.items():
        I, grid, nsplit = adaptive_quad(a, b, integrand, tol, M, method_func)
        results[method_name] = {
            "Approx Integral": I,
            "Grid Nodes": grid,
            "Interval Splits": nsplit
        }
        print(f"\nMethod: {method_name}")
        print(f"  Approximate Integral: {I}")
        print(f"  Number of Subinterval Splits: {nsplit}")
        print(f"  Unique Grid Nodes: {grid}")


Adaptive Quadrature Results for sin(1/x) on [0.1, 2] with tol=1e-3 and M=5 nodes:

Method: Composite Trapezoidal
  Approximate Integral: 1.1453764121229983
  Number of Subinterval Splits: 9
  Unique Grid Nodes: [0.1        0.10371094 0.10742188 0.11113281 0.11484375 0.11484375
 0.11855469 0.12226563 0.12597656 0.1296875  0.13339844 0.13710938
 0.14082031 0.14453125 0.14824219 0.15195312 0.15566406 0.159375
 0.159375   0.16679687 0.17421875 0.18164062 0.1890625  0.19648437
 0.20390625 0.21132813 0.21875    0.21875    0.23359375 0.2484375
 0.26328125 0.278125   0.29296875 0.3078125  0.32265625 0.3375
 0.35234375 0.3671875  0.38203125 0.396875   0.41171875 0.4265625
 0.44140625 0.45625    0.47109375 0.4859375  0.50078125 0.515625
 0.53046875 0.5453125  0.56015625 0.575      0.575      0.634375
 0.69375    0.753125   0.8125     0.871875   0.93125    0.990625
 1.05       1.109375   1.16875    1.16875    1.228125   1.2875
 1.346875   1.40625    1.465625   1.525      1.584375   1.64375
 1.703